## Testing for load magasinstatistikk to source database


In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine, Engine

from typing import Iterable

In [5]:
def load_raw_api_data(api_url: str) -> pd.DataFrame:
    """ Request GET data from url and store in dataframe """
    r = requests.get(api_url)
    df = pd.DataFrame(r.json())

    return df

### 1. Prepare areas for loading into db table "area":
### api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOmråder'

In [34]:
area_df = load_raw_api_data(
    api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOmråder'
)

area_df.head()


,land,elspot,vassdrag
0,"[{'navn': 'Norge', 'navn_langt': 'Norge', 'bes...","[{'navn': 'NO 1', 'navn_langt': 'Elspotområde ...","[{'navn': 'VASS1', 'navn_langt': 'Vassdragsomr..."


In [35]:
def prepare_areas(area_df: pd.DataFrame) -> pd.DataFrame:
    
    def normalize_and_assign_areas(area_df: pd.DataFrame, area: str) -> pd.DataFrame:
        return pd.json_normalize(area_df[area].iloc[0]).assign(current_area=area)

    areas = ['land', 'elspot', 'vassdrag']
    result_df = (
        pd.concat(map(lambda area: normalize_and_assign_areas(area_df, area), areas), ignore_index=True)
        .rename(columns={'omrType': 'omr_type'})
    )

    return result_df

area_df = prepare_areas(area_df)

area_df

,navn,navn_langt,beskrivelse,omr_type,omrnr,current_area
0,Norge,Norge,Hele landet,NO,0,land
1,NO 1,Elspotområde 1,Øst-Norge. Omfatter østlige del av Østlandet f...,EL,1,elspot
2,NO 2,Elspotområde 2,"Sørvest-Norge. Omfatter sørlige del av Viken, ...",EL,2,elspot
3,NO 3,Elspotområde 3,Midt-Norge. Omfatter nordre og vestlige del av...,EL,3,elspot
4,NO 4,Elspotområde 4,Nord-Norge. Omfatter resten av Trøndelag og No...,EL,4,elspot
5,NO 5,Elspotområde 5,Vest-Norge. Omfatter midtre del av Vestland op...,EL,5,elspot
6,VASS1,Vassdragsområde 1,"Sørøst-Norge. Østlandet, Agder-fylkene og dele...",VASS,1,vassdrag
7,VASS2,Vassdragsområde 2,"Vest-landet. Resten av Rogaland, mesteparten a...",VASS,2,vassdrag
8,VASS3,Vassdragsområde 3,"Midt-Norge. Møre og Romsdal, Trøndelag og sørl...",VASS,3,vassdrag
9,VASS4,Vassdragsområde 4,Nord-Norge. Resten av Nordland og nordover.,VASS,4,vassdrag


In [36]:
area_df.dtypes

navn            object
navn_langt      object
beskrivelse     object
omr_type        object
omrnr            int64
current_area    object
dtype: object

### 2. Retrieve magasin statistics data from:
### api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligData'
#### We do this before preparing dates to only get the necessary date data we need for the magasin statistics

In [6]:
magasin_df = load_raw_api_data(
    api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligData',
)

magasin_df['dato_Id'] = pd.to_datetime(magasin_df['dato_Id'], format='%Y-%m-%d', errors='coerce')
magasin_df['neste_Publiseringsdato'] = pd.to_datetime(magasin_df['neste_Publiseringsdato'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')

### 3. Prepare dates for loading into db table "dato_dimensjon": 

In [8]:
def prepare_dato_dimensjon(years: Iterable[int]) -> pd.DataFrame:
    """Create a DataFrame modeled "dato_dimensjon" with all date data for given years."""
     
    # Generate all dates for the given years
    date_range = pd.date_range(start=f"{min(years)}-01-01", end=f"{max(years)}-12-31")

    df = pd.DataFrame({
        "iso_dato": date_range,
        "iso_aar": date_range.year,
        "iso_uke": date_range.isocalendar().week,
        "iso_maaned": date_range.month,
        "iso_dag": date_range.day
    })
    
    df = df.reset_index(drop=True)

    return df

iso_aar = magasin_df['iso_aar'].unique()

dato_df = prepare_dato_dimensjon(iso_aar)
print(f"First 5: \n {dato_df.head(5)} \n \n Last 5: \n {dato_df.tail(5)} ")
print(iso_aar)

First 5: 
     iso_dato  iso_aar  iso_uke  iso_maaned  iso_dag
0 1995-01-01     1995       52           1        1
1 1995-01-02     1995        1           1        2
2 1995-01-03     1995        1           1        3
3 1995-01-04     1995        1           1        4
4 1995-01-05     1995        1           1        5 
 
 Last 5: 
         iso_dato  iso_aar  iso_uke  iso_maaned  iso_dag
11318 2025-12-27     2025       52          12       27
11319 2025-12-28     2025       52          12       28
11320 2025-12-29     2025        1          12       29
11321 2025-12-30     2025        1          12       30
11322 2025-12-31     2025        1          12       31 
[2007 2015 2005 2009 1995 2018 2016 2004 2003 2022 1997 2011 2012 2023
 2000 2020 2008 2002 2017 2024 1999 2019 1996 2010 2021 2001 1998 2013
 2006 2014 2025]


### 4. Drop 'iso_aar' and 'iso_uke' which provides unecessary memory

In [39]:
magasin_df = magasin_df.drop(columns=['iso_aar', 'iso_uke'])

### 5. Prepare magasin statistics for loading into db table "magasinstatistikk_model":

In [40]:
magasin_df.head()

,dato_Id,omrType,omrnr,fyllingsgrad,kapasitet_TWh,fylling_TWh,neste_Publiseringsdato,fyllingsgrad_forrige_uke,endring_fyllingsgrad
0,2007-03-04,EL,5,0.416593,17.425789,7.259468,NaT,0.450818,-0.034225
1,2015-02-08,EL,5,0.485430,17.425789,8.458999,NaT,0.527302,-0.041872
2,2005-09-11,EL,5,0.875464,17.425789,15.255660,NaT,0.858146,0.017319
3,2009-02-22,EL,5,0.391005,17.425789,6.813575,NaT,0.425169,-0.034164
4,1995-05-07,EL,5,0.172278,17.425789,3.002083,NaT,0.175498,-0.003219


In [41]:
magasin_df.shape

(14139, 9)

In [42]:
magasin_df.dtypes

dato_Id                     datetime64[ns]
omrType                             object
omrnr                                int64
fyllingsgrad                       float64
kapasitet_TWh                      float64
fylling_TWh                        float64
neste_Publiseringsdato      datetime64[ns]
fyllingsgrad_forrige_uke           float64
endring_fyllingsgrad               float64
dtype: object

In [43]:
magasin_df = magasin_df.rename(
    columns={
        'dato_Id': 'dato_id',
        'omrType': 'omr_type',
        'kapasitet_TWh': 'kapasitet_twh',
        'fylling_TWh': 'fylling_twh',
        'fyllingsgrad': 'fyllingsgrad',
        'neste_Publiseringsdato': 'neste_publiseringsdato'
    }
)
magasin_df.head()

,dato_id,omr_type,omrnr,fyllingsgrad,kapasitet_twh,fylling_twh,neste_publiseringsdato,fyllingsgrad_forrige_uke,endring_fyllingsgrad
0,2007-03-04,EL,5,0.416593,17.425789,7.259468,NaT,0.450818,-0.034225
1,2015-02-08,EL,5,0.485430,17.425789,8.458999,NaT,0.527302,-0.041872
2,2005-09-11,EL,5,0.875464,17.425789,15.255660,NaT,0.858146,0.017319
3,2009-02-22,EL,5,0.391005,17.425789,6.813575,NaT,0.425169,-0.034164
4,1995-05-07,EL,5,0.172278,17.425789,3.002083,NaT,0.175498,-0.003219


In [44]:
POSTGRES_USER="postgres"
POSTGRES_PASSWORD="changethis"

POSTGRES_SRC_SERVER="localhost"
POSTGRES_SRC_PORT=5433
POSTGRES_SRC_DB="source_nve_db"
SRC_DB_URL=f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_SRC_SERVER}:{POSTGRES_SRC_PORT}/{POSTGRES_SRC_DB}"


In [45]:
engine = create_engine(SRC_DB_URL)

### 6. Insert dates into "dato_dimensjon" table in db

In [ ]:
def insert_into_db(engine: Engine, *, table_name: str, df: pd.DataFrame) -> pd.DataFrame:
    """ 
    Insert data into database and return a DF with all rows from the table,
    including any new columns created by the database 
    """ 
    with engine.begin() as conn:
        df.to_sql(
            name=table_name,
            con=conn,
            if_exists='append',
            index=False
        )aggrega
        
        new_df = pd.read_sql_table( table_name=table_name,
            con=conn
        )
            
        return new_df

In [47]:
dato_dim_table_df = insert_into_db(engine, table_name='dato_dimensjon', df=dato_df)

In [48]:
dato_dim_table_df.head()

,id,iso_dato,iso_aar,iso_uke,iso_maaned,iso_dag
0,1,1995-01-01,1995,52,1,1
1,2,1995-01-02,1995,1,1,2
2,3,1995-01-03,1995,1,1,3
3,4,1995-01-04,1995,1,1,4
4,5,1995-01-05,1995,1,1,5


In [49]:
dato_dim_table_df.dtypes

id                     int64
iso_dato      datetime64[ns]
iso_aar                int64
iso_uke                int64
iso_maaned             int64
iso_dag                int64
dtype: object

### 7. Insert areas into "area" table in db

In [50]:
area_table_df = insert_into_db(engine, table_name='area', df=area_df)

In [51]:
area_table_df.head()

,id,navn,navn_langt,beskrivelse,omr_type,omrnr,current_area
0,1,Norge,Norge,Hele landet,NO,0,land
1,2,NO 1,Elspotområde 1,Øst-Norge. Omfatter østlige del av Østlandet f...,EL,1,elspot
2,3,NO 2,Elspotområde 2,"Sørvest-Norge. Omfatter sørlige del av Viken, ...",EL,2,elspot
3,4,NO 3,Elspotområde 3,Midt-Norge. Omfatter nordre og vestlige del av...,EL,3,elspot
4,5,NO 4,Elspotområde 4,Nord-Norge. Omfatter resten av Trøndelag og No...,EL,4,elspot


In [52]:
area_table_df.dtypes

id               int64
navn            object
navn_langt      object
beskrivelse     object
omr_type        object
omrnr            int64
current_area    object
dtype: object

### 8. Merge magasin with date_dim_table and area_table dataframes and adjust columns

In [53]:
magasin_df = (
    magasin_df
    .merge(
    right=dato_dim_table_df[['id', 'iso_dato']],
    how='left',
    left_on='dato_id',
    right_on='iso_dato',
    validate='m:1')
    .drop(columns=['dato_id'])
    .rename(columns={'id': 'dato_id'})
    .merge(
        right=area_table_df[['id', 'omr_type', 'omrnr']],
        how='left',
        left_on=['omrnr', 'omr_type'],
        right_on=['omrnr', 'omr_type'],
        validate='m:1'
    )
    .rename(columns={'id': 'area_id'})
    .drop(
        columns=['iso_dato', 'omrnr', 'omr_type']
    )
)
    

In [54]:
magasin_df.head()

,fyllingsgrad,kapasitet_twh,fylling_twh,neste_publiseringsdato,fyllingsgrad_forrige_uke,endring_fyllingsgrad,dato_id,area_id
0,0.416593,17.425789,7.259468,NaT,0.450818,-0.034225,4446,6
1,0.485430,17.425789,8.458999,NaT,0.527302,-0.041872,7344,6
2,0.875464,17.425789,15.255660,NaT,0.858146,0.017319,3907,6
3,0.391005,17.425789,6.813575,NaT,0.425169,-0.034164,5167,6
4,0.172278,17.425789,3.002083,NaT,0.175498,-0.003219,127,6


### 9. Insert magasin into "magasinstatistikk_model" table in db

In [55]:
magasin_table_df = insert_into_db(engine, table_name='magasinstatistikk_model', df=magasin_df)

### 10. Retrieve magasin statistics min max median model from:
### api_url: https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligDataMinMaxMedian

In [56]:
magasin_min_max_df = load_raw_api_data(
    api_url='https://biapi.nve.no/magasinstatistikk/api/Magasinstatistikk/HentOffentligDataMinMaxMedian'
)

In [57]:
magasin_min_max_df.head()

,omrType,omrnr,iso_uke,minFyllingsgrad,minFyllingTWH,medianFyllingsGrad,medianFylling_TWH,maxFyllingsgrad,maxFyllingTWH
0,EL,1,1,0.456893,2.742963,0.616703,3.702380,0.759600,4.560268
1,EL,1,2,0.422561,2.536850,0.573354,3.442137,0.698279,4.192127
2,EL,1,3,0.389071,2.335794,0.533293,3.201628,0.655389,3.934633
3,EL,1,4,0.350166,2.102222,0.489547,2.939000,0.603621,3.623846
4,EL,1,5,0.318725,1.913467,0.449146,2.696455,0.570686,3.426118


### 11. Prepare magasin statistics min max median model for loading into db

In [ ]:
magasin_min_max_df = (
    magasin_min_max_df
    .rename(columns={
        'omrType': 'omr_type',
        'minFyllingsgrad': 'min_fyllingsgrad',
        'minFyllingTWH': 'min_fylling_twh',
        'medianFyllingsGrad': 'median_fyllingsgrad',
        'medianFylling_TWH': 'median_fylling_twh',
        'maxFyllingsgrad': 'max_fyllingsgrad',
        'maxFyllingTWH': 'max_fylling_twh',
    })
    .merge(
    right=area_table_df[['id', 'omr_type', 'omrnr']],
    how='left',
    left_on=['omrnr', 'omr_type'],
    right_on=['omrnr', 'omr_type'],
    validate='m:1'
    )
    .rename(columns={'id': 'area_id'})
    .drop(
    columns=['omrnr', 'omr_type']
    )
)



In [60]:
magasin_min_max_df.head()

,iso_uke,min_fyllingsgrad,min_fylling_twh,median_fyllingsgrad,median_fylling_twh,max_fyllingsgrad,max_fylling_twh,area_id
0,1,0.456893,2.742963,0.616703,3.702380,0.759600,4.560268,2
1,2,0.422561,2.536850,0.573354,3.442137,0.698279,4.192127,2
2,3,0.389071,2.335794,0.533293,3.201628,0.655389,3.934633,2
3,4,0.350166,2.102222,0.489547,2.939000,0.603621,3.623846,2
4,5,0.318725,1.913467,0.449146,2.696455,0.570686,3.426118,2


### 12. Insert magasin statistics min max median into db

In [ ]:
magasin_min_max_table_df = insert_into_db(engine, table_name='magasinstatistikk_min_max_model', df=magasin_min_max_df)

In [62]:
magasin_min_max_table_df.head()

,iso_uke,min_fyllingsgrad,min_fylling_twh,median_fyllingsgrad,median_fylling_twh,max_fyllingsgrad,max_fylling_twh,area_id
0,1,0.456893,2.742963,0.616703,3.702380,0.759600,4.560268,2
1,2,0.422561,2.536850,0.573354,3.442137,0.698279,4.192127,2
2,3,0.389071,2.335794,0.533293,3.201628,0.655389,3.934633,2
3,4,0.350166,2.102222,0.489547,2.939000,0.603621,3.623846,2
4,5,0.318725,1.913467,0.449146,2.696455,0.570686,3.426118,2


# This seems ok for now